In [64]:
import os
import pandas as pd
from functools import reduce
import numpy as np
from datetime import date
import tensorflow as tf
from keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pmdarima as pm


In [65]:
df=pd.read_csv("C:/Users/91966/OneDrive/Desktop/capstone/telangana/weather/daily_weather_data.csv")
df=df[['district','odate','rainfall','temp_min','temp_max']].dropna()

In [66]:
df1=df.copy(deep=True)
df2=df.copy(deep=True)
df3=df.copy(deep=True)
df1['odate'] = pd.to_datetime(df1['odate'], format='%d-%m-%Y', errors='coerce')
df1.dropna(inplace=True)
df2['odate'] = pd.to_datetime(df2['odate'], format='%Y-%m-%d', errors='coerce')
df2.dropna(inplace=True)
df3['odate'] = pd.to_datetime(df3['odate'], format='%d-%b-%y', errors='coerce')
df3.dropna(inplace=True)

In [67]:
df_final=pd.concat([df1,df2,df3],axis=0).sort_index()
df_final=df_final.sort_values(by='odate')

In [68]:
df_comps={}
options=["Adilabad","Nizamabad","Warangal","Karimnagar","Khammam"]
for i in options:
  df_comps[i]=df_final[df_final["district"]==i]
district='Khammam'

In [69]:
df_comps[district]=df_comps[district][['odate','temp_max','temp_min']]

In [70]:
for i in df_comps:
    df_comps[i]=df_comps[i].groupby(by=["odate"]).mean()


C:\Users\91966\AppData\Local\Temp\ipykernel_20532\366426764.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_comps[i]=df_comps[i].groupby(by=["odate"]).mean()
C:\Users\91966\AppData\Local\Temp\ipykernel_20532\366426764.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_comps[i]=df_comps[i].groupby(by=["odate"]).mean()
C:\Users\91966\AppData\Local\Temp\ipykernel_20532\366426764.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid

In [71]:
df_comps['Adilabad']

,rainfall,temp_min,temp_max
odate,,,
2018-01-01,0.000000,11.200000,30.436842
2018-01-02,0.000000,11.810526,31.173684
2018-01-03,0.000000,11.626316,31.121053
2018-01-04,0.000000,11.200000,30.300000
2018-01-05,0.000000,10.700000,29.410526
...,...,...,...
2022-09-26,0.000000,23.544444,33.972222
2022-09-27,11.888889,24.683333,33.350000
2022-09-28,1.166667,23.661111,33.894444


In [72]:

df=df_comps[district]

df["avg_temp"] = ((df["temp_max"] + df["temp_min"])/2)

In [73]:
holt_winter = sm.tsa.ExponentialSmoothing(
    df['temp_max'], seasonal='add', seasonal_periods=365)
x = holt_winter.fit()
x.params

c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


{'smoothing_level': 0.720869751982435,
 'smoothing_trend': nan,
 'smoothing_seasonal': 1.0528679006976707e-08,
 'damping_trend': nan,
 'initial_level': 36.8712864456577,
 'initial_trend': nan,
 'initial_seasons': array([-3.78265625, -3.79178404, -3.49984383, -4.02288777, -3.81845216,
        -3.41437033, -3.9042205 , -3.91566335, -3.35674622, -3.46089361,
        -3.05063765, -2.77944207, -2.98798961, -3.65530161, -3.52684577,
        -2.90918739, -2.66974554, -3.1186217 , -2.83873417, -2.63156008,
        -2.4509206 , -2.21009719, -2.53270058, -2.14898627, -2.18932511,
        -2.27919591, -2.26849557, -3.20661636, -3.64613008, -2.94834452,
        -2.41853021, -1.27010948, -1.45488703, -1.38209029, -1.45520749,
        -1.71810749, -1.40730149, -1.15625041, -2.17117259, -2.27840959,
        -2.04225556, -1.32638187, -1.879363  , -1.08866773, -0.65532614,
        -0.21421038, -0.35476911, -0.30564778, -0.28260331, -0.46787226,
        -1.10389865, -0.42367936,  0.30224802,  1.07442821

In [74]:
holt_winter1 = sm.tsa.ExponentialSmoothing(
    df['avg_temp'], seasonal='add', seasonal_periods=365)
y = holt_winter1.fit()
y.params

c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


{'smoothing_level': 0.7988356620600764,
 'smoothing_trend': nan,
 'smoothing_seasonal': 5.532558863687082e-10,
 'damping_trend': nan,
 'initial_level': 30.350705890624578,
 'initial_trend': nan,
 'initial_seasons': array([-4.67698514, -4.6296859 , -4.33022723, -4.85674761, -4.65965976,
        -4.07213124, -4.41329822, -4.4084643 , -4.44927192, -4.30164897,
        -3.80795141, -3.42311353, -3.33823052, -3.7579597 , -3.5346029 ,
        -3.37590564, -3.22364458, -3.61900252, -3.82246963, -3.83995897,
        -3.67662211, -3.3898413 , -3.1541378 , -2.72382796, -3.218712  ,
        -3.32191586, -2.76343893, -3.73733145, -4.2746935 , -4.19974472,
        -3.79071446, -3.89535679, -4.2171753 , -3.77104531, -3.92393036,
        -4.78858873, -4.10736224, -3.62642699, -3.22666852, -2.91455702,
        -3.18502305, -2.61129205, -3.6301472 , -3.52930883, -3.62552468,
        -2.96213281, -2.32542684, -2.50772442, -1.71782131, -2.08262314,
        -2.35568418, -2.7803619 , -2.25259695, -1.820502

In [75]:
df['temp_max_pred'] = list(x.predict(1, len(df)))
# df[['aqi', 'predict_holt']].plot()

df1=pd.DataFrame(x.forecast(365))
df1.columns=['temp_max_pred']
df1

    

c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


,temp_max_pred
1733,33.855801
1734,34.221124
1735,34.905764
1736,34.993412
1737,34.125874
...,...
2093,33.453813
2094,32.359680
2095,32.665341
2096,33.550998


In [76]:
def prob(mae_tempmax,mae_tempavg,temp_max,temp_avg):
    final_temp_max=temp_max-mae_tempmax
    final_temp_avg=temp_avg+mae_tempavg
    prob=min((final_temp_max-final_temp_avg)/4.5,0.99)*100
    prob_severe=min((final_temp_max-final_temp_avg)/6.4,0.99)*100
    return prob,prob_severe

In [77]:
df['temp_avg_pred'] = list(y.predict(1, len(df)))
# df[['aqi', 'predict_holt']].plot()
df2=pd.DataFrame(y.forecast(365))
df2.columns=['temp_avg_pred']
df_final=pd.concat([df1,df2],axis=1)
df_final

c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


,temp_max_pred,temp_avg_pred
1733,33.855801,28.502153
1734,34.221124,29.245976
1735,34.905764,29.647189
1736,34.993412,29.625262
1737,34.125874,28.818397
...,...,...
2093,33.453813,28.563435
2094,32.359680,27.739506
2095,32.665341,28.412274
2096,33.550998,28.845738


In [78]:
df_final['Normal_heat_wave_prediction']=df_final.apply(lambda x:prob(0.4,0.2,x['temp_max_pred'],x['temp_avg_pred'])[0],axis=1)
df_final['Severe_heat_wave_prediction']=df_final.apply(lambda x:prob(0.4,0.2,x['temp_max_pred'],x['temp_avg_pred'])[1],axis=1)

In [79]:
df_final

,temp_max_pred,temp_avg_pred,Normal_heat_wave_prediction,Severe_heat_wave_prediction
1733,33.855801,28.502153,99.000000,74.275748
1734,34.221124,29.245976,97.225515,68.361690
1735,34.905764,29.647189,99.000000,72.790242
1736,34.993412,29.625262,99.000000,74.502346
1737,34.125874,28.818397,99.000000,73.554330
...,...,...,...,...
2093,33.453813,28.563435,95.341738,67.037160
2094,32.359680,27.739506,89.337200,62.815219
2095,32.665341,28.412274,81.179270,57.079174
2096,33.550998,28.845738,91.227990,64.144681


In [80]:
from pathlib import Path
save_location = Path(f"pred-{district}")
x.save(fname=save_location.absolute())